In [2]:
#r "nuget:XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

Installed Packages XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Loading extensions from `/mnt/silo/hts2024/alehmann/.nuget/packages/xplot.plotly.interactive/4.0.7/lib/net7.0/XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [3]:
using XPlot.Plotly;
using System.Text.Json;
using System.IO;

### Exercise 3.2.1

- Per Tile Sequence Quality: shows, whether a part of the flow cell is probably damaged
- Per Sequence GC Content --> assumption roughly binomial distribution of GC content in reads. When not binomial distribution then contaminated or other biased subset. 
- Per base sequence quality, illustrated in box whisker diagram with error bars --> Trimmomatic to cut low quality reads
- Per sequence quality scores --> Mean quality score of sequence read vs count of reads, which have the corresponding quality score

#### 3.2.1.3
- [SQUAT](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-5445-3)
- [GenomeQC](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-020-6568-2)
- [FASTQuick](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7844880/) and PIQA, HTQC, QPLOT, Picard, verifyBamID2

### Exercise 3.2.2

- Trimmomatic trims sequence reads based upon the following quality criteria:
  - LEADING: Cut bases of the start of a read, when below threshold quality
  - TRAILING: Same from end and severall other cropping techniques
  - it can also convert quality values
- `java -jar trimmomatic-0.39.jar SE -phred33 ~/../data/Capsella-sequencing/DNA-seq/rawdata/DNA1_1.fq.gz ~/hts-exercises/3-2/DNA1_1_trimmed.fq.gz TRAILING:28` --> phred33 is sanger, and wrong for this case
- SE means Single End mode and PE means Paired end mode. Paired end mode results in 4 output files, reverse and normal direction read permutated with paired and unpaired. Paired means, that the read in both direction is preserved. Unpaired means that one read of one direction got discarded because of quality criteria
- You can remove adapters with trimmomatic, which are in the folder adapters of trimmomatic. Adapters are used for primers on each flank of the DNA sequence after breaking up. An adapter consists of:
  - Flow cell binding sequence, which is platform specific for the flow cell
  - Sequencer Primer binding site, which allows binding of sequencing primers for polymerases to bind and extend the synthesis
  - Tag or index regions are used to make analysis on flow cells as cheap as possible. Prepare multiple DNA samples with differently indexed adapters and then you can PCR them on the flow cell and demultiplex them into your samples 
- `fastqc ~/hts-exercises/3-2/DNA1_1_cropped.fq.gz -o ~/hts-exercises/3-2/fastqcResult/
application/gzip`
- Sickle is another Tool

### Exercise 3.2.3

1. Trimmomatic with the option TOPHRED33
2. `java -jar ~/libs/trimmomatic/Trimmomatic-0.39/trimmomatic-0.39.jar SE ~/../data/Capsella-sequencing/DNA-seq/rawdata/DNA1_1.fq.gz ~/hts-exercises/3-2/DNA1_1_sanger.fq.gz TOPHRED33`
3. `zcat ~/hts-exercises/3-2/DNA1_1_sanger.fq.gz | awk "NR%4==2 || NR%4==1"| sed "s/@/>/g" | gzip > ~/hts-exercises/3-2/DNA1_1_sanger.fa.gz` sed is enough, because no quality data is included

### Exercise 3.2.4

1. Assumption: subreads of fixed length are shared between reads --> Problematic when sequencing a mixed population with variants
   1. Zusätzlich Multiple sequence alignment um häufige k-mere zu finden --> 3 Methoden: k-spectrum based, suffix tree/array-based, MSA based
   2. K-spectrum based:
      1. Align common k-mers to each other, which have a small Hamming distance from each other and make a consensus of the k-mer sequence	
      2. k-spectrums can be called solid, when a set occurs at least M times --> insolid otherwise. Insolid ones are then converted to solid ones. SOAPdenovo
      3.  Quake: identify solid and insolid k-mers and use probabilistic methods to find solid ones
  1.  SHREC: Tree-based use multiple k and M values to find solid/insolid k-mers
  2.  MSA-based Coral make MSA and majority voting
  3.  BFC seems to work pretty well and tries to find a k-mer path through a read to minimize cost function. It applies penalties on correction, quality and k-mer support. 

### 3.2.5

1. Description in the folder hts-exercises, grouped by exercise folder. In the exercise folder is the processed sequencing data. Programs are linked in a bin folder and programs to compile in the libs folder. 
2. This one is actually quite good. Everything in the same folder would be annoying. External servers are overkill for just testing and screwing around
3. Storing the markdown files on github as a summary would be interesting for future documentation

### 3.2.6

1. Sell your soul to a company to store the data for you. Else public databases like ncbi
   1. NCBI SRA, dbSNP, 
   2. DDBJ Data Bank of Japan
   3. ENA European Nucleotide Archive
   4. China National Gene Bank Database
   5. Illumina BaseSpace
   6. Only NCBI SRA, ENA and DDBJ hold really genomes --> The synchronize with each other
2. Access the Sequence Read Archive (SRA) in NCBI instead
3. Search in NCBI SRA for `(potsdam) AND capsella`

### 3.2.7

1. Raw sequencing data were deposited in the European Nucleotide Archive (ENA) under project PRJEB21270. Search for `PRJEB21270` in ENA website
2. Illumina Sequencing with TruSeq DNA PCR-Free Library Preparation Kit, S2 Focused-Ultrasonicator, Illumna MiSeq using 500-cycle MiSeq Reagent Kit v2
3. fastq.gz files
4. Do FastQC and maybe trimmomatic or bcf